<a href="https://colab.research.google.com/github/tsm-mehmetakiftasoz/tsm_makif/blob/main/Z.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

# ChatGPT (OpenAI API) - Colab Widget Chat
# =========================

!pip install -q openai ipywidgets

import re, time
import ipywidgets as widgets
from IPython.display import display, HTML
from google.colab import output
from openai import OpenAI

# -------------------------
# UI: Connection status + chat area
# -------------------------
baglanti_durumu = widgets.HTML("<b style='color:#ff9800;'>🔄 OpenAI'ye bağlanmaya hazır...</b>")

output_area = widgets.Output(layout={
    'border': '3px solid #2b6cb0',
    'border_radius': '10px',
    'padding': '10px',
    'background_color': '#f7fafc',
    'max_height': '400px',
    'overflow_y': 'auto'
})

api_key_box = widgets.Password(
    placeholder="🔑 OpenAI API Key (sk-... veya projene ait anahtar)",
    layout=widgets.Layout(width='80%')
)
connect_btn = widgets.Button(
    description='🔌 Bağlan',
    button_style='info',
    layout=widgets.Layout(width='20%')
)

metin_kutusu = widgets.Textarea(
    placeholder='💬 Mesaj yaz... (Enter = yeni satır, Ctrl+Enter = gönder; bitirmek için "mischief managed.")',
    layout=widgets.Layout(width='80%', height='80px', padding='5px'),
    disabled=True
)
gonder_butonu = widgets.Button(
    description='🚀 Gönder',
    button_style='success',
    layout=widgets.Layout(width='20%', height='80px', padding='5px'),
    disabled=True
)

ui = widgets.VBox([
    baglanti_durumu,
    widgets.HBox([api_key_box, connect_btn]),
    output_area,
    widgets.HBox([metin_kutusu, gonder_butonu])
])
display(ui)

# -------------------------
# OpenAI client + conversation memory
# -------------------------
client = None
conversation_history = [] # list of {"role": "...", "content": "..."}
MODEL_NAME = "gpt-5.2" # İstersen "gpt-4.1" gibi başka bir model adıyla değiştir.

# -------------------------
# Message bubble renderer (Markdown **bold** -> <b>)
# -------------------------
def mesaj_goster(kimden, metin):
    metin_html = re.sub(r'\*\*(.*?)\*\*', r'<b>\1</b>', metin)
    renk = '#9ae6b4' if kimden == 'Siz' else '#bee3f8'
    hizalama = 'right' if kimden == 'Siz' else 'left'
    with output_area:
        display(HTML(f"""
<div class='msg' style='text-align:{hizalama}; margin: 6px 0;'>
<div style='display:inline-block; background:{renk}; border-radius:10px; padding:8px 12px; max-width:80%; white-space: pre-wrap;'>
<b>{kimden}:</b> {metin_html}
</div>
</div>
"""))

# -------------------------
# End chat helpers
# -------------------------
def sohbeti_kapat():
    metin_kutusu.disabled = True
    gonder_butonu.disabled = True
    baglanti_durumu.value = "<b style='color:#805ad5;'>✨ Çat kapandı. Marauder's Map kapanıyor... 👁️</b>"

def fade_out_mesajlar():
    display(HTML("""
<script>
let msgs = document.querySelectorAll('.msg');
msgs.forEach(m => m.style.transition = 'opacity 2s');
setTimeout(() => { msgs.forEach(m => m.style.opacity = 0); }, 500);
</script>
"""))

# -------------------------
# Connect
# -------------------------
def connect(_=None):
    global client
    key = (api_key_box.value or "").strip()
    if not key:
        baglanti_durumu.value = "<b style='color:red;'>❌ API key boş. Lütfen kendi OpenAI API key'ini gir.</b>"
        return

    try:
        baglanti_durumu.value = "<b style='color:#ff9800;'>🔄 OpenAI'ye bağlanılıyor...</b>"
        client = OpenAI(api_key=key)
        # Basit bir ping: kısa yanıt iste
        _ = client.responses.create(model=MODEL_NAME, input="Say 'connected'.")
        time.sleep(0.6)
        baglanti_durumu.value = "<b style='color:#38a169;'>✅ Başarılı bağlantı.</b><br><i>\"I solemnly swear that I am up to no good.\"</i>"
        metin_kutusu.disabled = False
        gonder_butonu.disabled = False
    except Exception as e:
        client = None
        baglanti_durumu.value = f"<b style='color:red;'>❌ Bağlantı hatası: {e}</b>"

connect_btn.on_click(connect)

# -------------------------
# Send message
# -------------------------
def mesaj_gonder(_=None):
    global conversation_history, client
    if client is None:
        mesaj_goster("Sistem", "Önce 🔌 Bağlan butonuyla API'ye bağlan.")
        return

    kullanici_mesaji = (metin_kutusu.value or "").strip()
    if not kullanici_mesaji:
        return

    mesaj_goster("Siz", kullanici_mesaji)
    metin_kutusu.value = ''

    # End chat phrase
    if kullanici_mesaji.lower() == "mischief managed.":
        fade_out_mesajlar()
        sohbeti_kapat()
        metin_kutusu.placeholder = 'Çat kapandı. 👁️'
        time.sleep(2.2)
        output_area.clear_output()
        metin_kutusu.placeholder = ' '
        return

    try:
        conversation_history.append({"role": "user", "content": kullanici_mesaji})

        # Responses API: her istekte geçmişi de gönderiyoruz (senin Gemini örneği gibi)
        resp = client.responses.create(
            model=MODEL_NAME,
            input=conversation_history
        )

        text_response = (resp.output_text or "").strip()
        if not text_response:
            text_response = "[Boş yanıt döndü]"

        mesaj_goster("ChatGPT", text_response)
        conversation_history.append({"role": "assistant", "content": text_response})

    except Exception as e:
        mesaj_goster("Sistem", f"Hata: {e}")

gonder_butonu.on_click(mesaj_gonder)

# -------------------------
# Ctrl+Enter = send (Colab JS callback)
# -------------------------
def js_klavye_dinleyici():
    display(HTML("""
<script>
document.querySelector('textarea').addEventListener('keydown', function(e) {
    if (e.ctrlKey && e.key === 'Enter') {
        google.colab.kernel.invokeFunction('notebook.mesajGonder', [], {});
        e.preventDefault();
    }
});
</script>
"""))

output.register_callback('notebook.mesajGonder', mesaj_gonder)
js_klavye_dinleyici()
